# Radial Basis Network

In [4]:
import pandas as pd
import numpy as np

from keras.layers import Layer
from keras import backend as K
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.losses import binary_crossentropy

In [1]:
class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = K.cast_to_floatx(gamma)

    def build(self, input_shape):
#         print(input_shape)
#         print(self.units)
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff, 2), axis=1)
        res = K.exp(-1 * self.gamma * l2)
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

# Custom non-trainable keras layers

In [2]:
X = np.load('/content/k49-train-imgs.npz')['arr_0']
y = np.load('/content/k49-train-labels.npz')['arr_0']
y = (y <= 25).astype(int)

model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(RBFLayer(10, 0.5))
model.add(Dense(1, activation='sigmoid', name='foo'))

model.compile(optimizer='rmsprop', loss=binary_crossentropy)

In [3]:
model.fit(X, y, batch_size=256, epochs=3)

Epoch 1/3
908/908 [==============================] - 18s 19ms/step - loss: 0.6821
Epoch 2/3
908/908 [==============================] - 16s 18ms/step - loss: 0.6805
Epoch 3/3
908/908 [==============================] - 17s 18ms/step - loss: 0.6805
